# Image Recognition with OpenCV

Based on [this blog post](http://www.pyimagesearch.com/2017/08/21/deep-learning-with-opencv/).

[OpenCV](http://opencv.org) installed using [this script](https://github.com/massenz/HOW-TOs/blob/master/OpenCV-Install.sh).

Some simple OpenCV C++ code can be found in this repository in the `opencv/DisplayImage.cpp` file.



In [44]:
import argparse
import csv
import cv2
import numpy as np
import os
import time

# Constants

IMAGE = 'data/fox.jpg'

# https://github.com/opencv/opencv_extra/blob/master/testdata/dnn/bvlc_googlenet.prototxt
PROTOTXT = 'data/bvlc_googlenet.prototxt'

# https://github.com/BVLC/caffe/tree/master/models/bvlc_googlenet
MODEL_FILE = 'data/bvlc_googlenet.caffemodel'

# https://github.com/HoldenCaulfieldRye/caffe/blob/master/data/ilsvrc12/synset_words.txt
LABELS_FILE = 'data/synset_words.txt'

# Load the input image from disk
# Our CNN requires fixed spatial dimensions for our input image(s)
# so we need to ensure it is resized to 224x224 pixels while
# performing mean subtraction (104, 117, 123) to normalize the input;
# after executing this command our "blob" now has the shape:
# (1, 3, 224, 224)
image = cv2.imread(IMAGE)
blob = cv2.dnn.blobFromImage(image, 1, (224, 224), (104, 117, 123))

# Load the class labels from disk
rows = open(LABELS_FILE).read().strip().split("\n")
classes = [r[r.find(" ") + 1:].split(",")[0] for r in rows]


In [45]:
classes[:5]

['tench', 'goldfish', 'great white shark', 'tiger shark', 'hammerhead']

In [46]:
# load our serialized model from disk
print("[INFO] loading model...", end='')
net = cv2.dnn.readNetFromCaffe(PROTOTXT, MODEL_FILE)
print(" done.")

[INFO] loading model... done.


In [47]:
# set the blob as input to the network and perform a forward-pass to
# obtain our output classification
net.setInput(blob)
start = time.time()
preds = net.forward()
end = time.time()
print("[INFO] classification took {:.5} seconds".format(end - start))

# sort the indexes of the probabilities in descending order (higher
# probabilitiy first) and grab the top-5 predictions
idxs = np.argsort(preds[0])[::-1][:5]


[INFO] classification took 0.052839 seconds


In [48]:
# loop over the top-5 predictions and display them
for (i, idx) in enumerate(idxs):
    # draw the top prediction on the input image
    if i == 0:
        text = "Label: {}, {:.2f}%".format(classes[idx], preds[0][idx] * 100)
        cv2.putText(image, text, (5, 30), cv2.FONT_HERSHEY_SIMPLEX,
            1.2, (0, 0, 255), 2)
 
    # display the predicted label + associated probability to the
    # console
    print("[INFO] {}. label: {}, probability: {:.5}".format(i + 1,
        classes[idx], preds[0][idx]))
 
# display the output image
cv2.imshow(classes[idxs[0]], image)
cv2.waitKey(0)

[INFO] 1. label: wallaby, probability: 0.49319
[INFO] 2. label: miniature schnauzer, probability: 0.048758
[INFO] 3. label: coyote, probability: 0.027008
[INFO] 4. label: West Highland white terrier, probability: 0.026041
[INFO] 5. label: Siberian husky, probability: 0.025412


-1